<div style="text-align:center;">
    <img src="http://www.cs.wm.edu/~rml/images/wm_horizontal_single_line_full_color.png">
    <h1>CSCI 312-01, Fall 2025</h1>
    <h1>Effective C, Chapter 6</h1>
    <h1>Dynamically allocated memory</h1>
</div>

# Contents

* [Indirection](#Indirection)
* [More about pointers and addresses](#More-about-pointers-and-addresses)
* [A swap function](#A-swap-function)
* [Pointer arithmetic](#Pointer-arithmetic)
* [Pointers and arrays and strings](#Pointers-and-arrays-and-strings)
* [Bubblesort in C](#Bubblesort-in-C)
* [Pointers and <code class="kw">const</code>](#const)
* [Pointers in CPython](#Pointers-in-CPython)
* [Function pointers](#Function-pointers)
* [Dynamic memory allocation](#Dynamic-memory-allocation)

# Python vs. C vs. C++ vs. Java

|           | Python          | C            | C++   | Java  |
| :--------:| :-------------: | :----------: | :---: | :---: |
| pointer   |                 | type &#42;  | type &#42; |   |
| address of ```var``` |      | ```&var```  | ```&var``` |   |
| memory allocation |  | `malloc()`, `calloc()` | `new` | same as C++ |
| memory deallocation | | `free()` | `delete` | same as C++ |

# Indirection

<blockquote>
By indirections find directions out. <br/>
&ndash; Hamlet, Act 2, Scene 1
</blockquote>

In computer science, [**indirection**](https://en.wikipedia.org/wiki/Indirection) is the ability to refer to an object, well, indirectly through some mechanism other than the object itself.

Here is an illustration.  Imagine we need to sort a set of numbers in ascending order.  One simple approach (which does not scale well!) is **bubblesort**: we make repeated passes through the numbers, and if two adjacent numbers are out of order relative to one another we swap them.  We are done when we make a pass without finding any out of order numbers:

In [ ]:
cat -n src/bubblesort.py

In [ ]:
python3 src/bubblesort.py

Now suppose that instead of sorting numbers, we wish to sort large, heavy rocks so that when we are done they are lined up in order of ascending weight.  The swaps of adjacent rocks would be onerous work, so instead we will use indirection:
1.  First label the rocks (in no particular order) $1$ to $n$.
2.  For each rock, take a piece of paper and write the rock's weight and the label we gave it in step 1.
3.  Perform bubblesort on the pieces of paper so they are sorted in order of increasing weight.
4.  Once we have things sorted in order of ascending weight, look up the rocks using the labels from step 1 and assemble the rocks in sorted order.

The use of the labels for the rocks rather than the rocks themselves for the purposes of sorting is an example of indirection.  In this example it allows us to move the heavy rocks exactly one time each.

# More about pointers and addresses

One of C's strengths is the ability manipulate objects using indirection via their addresses in memory.  A [**pointer**](https://en.wikipedia.org/wiki/Pointer_(computer_programming)) is a variable that can hold a memory address.  They are declared with the syntax
```
type *var;
```
where ```type``` is the type of object we are pointing to (e.g., ```int```, ```float```).

For example:
```
int n;   // An integer.
int *p;  // A pointer to an integer.
int *q;  // Another pointer to an integer.
double *x  // A pointer to a double.
```

The `*` modifies the variable that follows, **not** the type that precedes it.  That is, the declaration is

<code>int* p, q;</code>
is equivalent to

<code>int *p, q;</code>

so
* <code>p</code> is a pointer to an <code>int</code>, but 
* <code>q</code> is an <code>int</code>.

<div class="danger"></div> 
🐞 Be careful &ndash; the <code>*</code> modifies the variable that follows, not the type that precedes it. 🐞

We can get the address of an existing variable with the `&` operator.

Let's take a look at some code:

In [ ]:
cat -n src/pt_intro.c

In [ ]:
clang -Wall src/pt_intro.c

In [ ]:
./a.out

At line 11 we set ```p``` to the address of ```n```.  We say that `p` now points to `n`.  We print the value of the pointer in line 12 using the `%p` print format.  The address begins with `0x`, which indicates that this is a hexadecimal (base 16) number.  The hexadecimal digits are
```
hexadecimal: 0 1 2 3 4 5 6 7 8 9 a  b  c  d  e  f
decimal:     0 1 2 3 4 5 6 7 8 9 10 11 12 13 14 15
```

We can access the value of the variable ```n``` through the pointer ```p``` by **dereferencing** the pointer.  The syntax ```*p``` in line 14 means the value at the memory location ```p``` points to.

Since ```p``` is declared to point to an ```int```, the statement ```*p``` says to

1. go to the location in memory stored in ```p```,
2. read one ```int``` worth of bits (typically 32 bits), and
3. interpret those bits as an ```int```.

In line 16 we write to the location that ```p``` points to, thereby changing `n`.

Multiple pointers can point to the same location, as lines 20 and 21 demonstrate.

🐞🐞 Pointers can be confusing. 🐞🐞


If <code>p</code> is a pointer, <code>p</code> refers to a location in memory.

In a declaration <code>&#42;</code> means the variable is a pointer.  When used in dereferencing, on the other hand, the <code>*</code> means the object the pointer points to, if it appears on the left-hand side of an assignment, or the value of the object the pointer points to, otherwise.

<pre>
double x = 54;   /* A double. */
double *p = &x;  /* A pointer to x. */
double y;        /* Another double. */

y = *p;   /* Equivalent to y = x. */
*p = 42;  /* Equivalent to x = 42. */
    
p = 42;   /* Sets p to point to the address 42, which is probably invalid! */
</pre>

Recall **references** from Python &ndash; these are two variables that refer to same underlying object.  In the following Python code, both `a` and `b` refer to the same list:

In [ ]:
cat -n src/reference.py

In [ ]:
python src/reference.py

References are like pointers, only without the need to dereference a pointer.  But references are implemented in practice using pointers that point to the same object in memory.

In [ ]:
./a.out

In hexadecimal `c` represents decimal 12:

```
hexadecimal: 0 1 2 3 4 5 6 7 8 9 a  b  c  d  e  f
decimal:     0 1 2 3 4 5 6 7 8 9 10 11 12 13 14 15
```

Suppose
```
p + 1 = 0x7ff7b34657dc
p     = 0x7ff7b34657d8
```
Then
```
(p + 1) - p = 0x7ff7b34657dc - 0x7ff7b34657d8 = c - 8 = 4.
```
or 4 bytes, which is the size of an `int`.

This illustrates an important aspect of working with pointers &ndash; usually we don't care about the values of pointers but rather the relative locations of pointers amd how far apart the addresses they contain are.

# Pointers and arrays and strings

There is a very close connection in C between pointers, arrays, and strings (which are just arrays of characters).  You should study the idioms in this section closely, as they are commonly encountered in C code.

First we look at the pointers and arrays:

In [ ]:
cat -n src/arrays.c

In line 11 we use array indexing to access the elements of the array `numbers`.

In line 16 we set the integer pointer `p` to equal `numbers`.  Since `numbers` is an array, thie has the effect of setting `p` to point to the beginning of `numbers` in memory.

In line 17, `*(p + i)` refers to the value `int` that is `i` integers past the beginning of the array `numbers`:
* `p + i` points to the location that is `i * sizeof(int)` bytes past the beginning of `numbers`, and
* `*(p + 1)` dereferences the address `p + i` as an `int`.  It is equivalent to `p[i]`.

In line 23 we achieve the same effect with the pointer `p` using array indexing, just as in line 11.

In [ ]:
gcc src/arrays.c

In [ ]:
./a.out

This example illustrates an important feature of C: **arrays are really just pointers (and vice-versa)**.

Arrays are not first-class objects in C &ndash; an array is really just a pointer to the beginning of the array.  The type of the pointer (e.g. `int*`, `double*`) determines how the bytes at that location and subsequent locations are interpreted.  You are responsible to keeping track of the length of the array.

<img src="https://www.cs.wm.edu/~rml/images/danger.svg" style="height: 30px"/> <img src="https://www.cs.wm.edu/~rml/images/danger.svg" style="height: 30px"/> 🐞 C hackers **love pointers** 💘💘, as they allow them to write terse and cryptic (but powerful!) code. 🐞

Here are some other C idioms that implement the loops we just looked at:

In [ ]:
cat -n src/arrays2.c

In line 17, `numbers + 10` means the address 10 `int` past the start of `numbers`.  In line 18 we encounter `*p++`.  This is evaluated as
* use the value of the `int` that `p` points to (`*p`), and
* then increment `p` (`p++`); i.e., advance `p` to point to the next `int`.

The `for` loop that begins at line 25 is equivalent to the `while` loop at line 18.

<div class="danger"></div><div class="danger"></div>
In general, incrementing a pointer advances it by the size (in bytes) of the object it points to, while decrementing a pointer moves it backward by the size of the object it points to.

To further illustrate C pointer idioms, here is a function that determines the length of a string.

A string in C is an array of characters terminated by an ASCII NULL character.  This means that strings in C are really just pointers of type `char*` that point to the first character in the string.  To read the string you go to that starting address and read bytes until you encounter the terminating ASCII NULL.

In [ ]:
cat -n src/strlen.c

In [ ]:
gcc src/strlen.c

In [ ]:
./a.out

Let's break down how this function works.  All the work is done in the ```for``` loop on line 6:
<code>
    for (count = 0; &#42;s; s++, count++);
</code>
This is equivalent to
<code>
count = 0;
while (*s) {
    s = s + 1;  // Advance one character.
    count = count + 1;  // Increment the counter.
}
</code>
We start with the pointer ```s``` at the beginning of the string; the increment ```s++``` moves us across the string.  We use the fact that <code>&#42;s</code> will not be zero until we reach the ASCII NULL that terminates the string so the non-zero value <code>&#42;s</code> is interpreted as "true" and the loop continues.

<b>Question.</b>  Why can't we declare ```count``` inside the ```for``` statement:
<code>
int strlen(char *s)
{
    for (int count = 0; *s; s++, count++);
    return count;
}
</code>

<b>Answer</b> <br/>
<div class="voila">
This would make <tt>count</tt> local to the <tt>for</tt> statement so it would not be available to return.
</div>

Is the following valid in C?

In [ ]:
cat -n src/string_sum.c

<b>Answer</b> <br/>
<div class="voila">
Yes.  We are adding 42 to a pointer, which is valid.  The sum <tt>s + 42</tt>refers to the address 42 characters (presumably bytes) past <tt>s</tt>code>
</div>

In [ ]:
clang -Wall -pedantic src/string_sum.c

In [ ]:
./a.out

# Bubblesort in C

As another example we will implement the bubblesort algorithm in C:

In [ ]:
cat -n src/bubblesort.c

Note that we use a <code class="kw">do</code> rather than a <code class="kw">while</code> loop as we did in Python.  I like this structure since it allows us to avoid initializing `not_done` outside the loop.

In [ ]:
clang -Wall src/bubblesort.c

In [ ]:
./a.out

# Pointers and <code class="kw">const</code> <a id="const"/>

When applied to pointers, the <code class="kw">const</code> storage class can refer either to
* a pointer,
* the value(s) it points to, or
* both.

Consider the following:

In [ ]:
cat -n src/const.c

When we attempt to compile the code, we see different sets of valid and invalid statements, depending on the <code class="kw">const</code> declarations in the calling sequences:

In [ ]:
clang -c src/const.c

<img src="https://www.cs.wm.edu/~rml/images/danger.svg" style="height: 30px"/> In C, pointer errors are a common source of mischief.  Program defensively and use the most restrictive declaration of pointers that you can.

# Pointers in CPython

CPython, the reference implementation of Python, is written in C and uses pointers extensively.

For instance, the Python `bubblesort()` function above takes a list `a` as its input and sorts it, and the sorting persists after we return from the function.  This is similar to the swap function we just looked at.  CPython is passing a pointer to the list to `bubblesort()` and that allows us to manipulate the actual list in memory rather than a copy of the list.

[CPython lists are actually arrays of pointers to Python objects](https://docs.python.org/2/faq/design.html#how-are-lists-implemented-in-cpython).  This makes list references extremely efficient, since accessing, say, `a[42]` is a matter of going directly to the 42nd entry in an array of pointers and then dereferencing the pointer.  If we used a true list then we would need to travel through the objects `a[0]` through `a[41]` before we could access `a[42]`.  The fact that we are storing pointers to Python objects rather than the objects themselves means we are storing values of the same type in the array so lists can be represented by heterogeneous types of objects.

CPython also handles all the memory management for you.  In particular, CPython manages [garbage collection](https://en.wikipedia.org/wiki/Garbage_collection_(computer_science)), which is the reclamation of memory from variables that are no longer in use.

# Function pointers

If you wish to pass a function to a procedure you will need to pass a pointer to the function.

This is done, for instance, in the *nix system sorting routines, where we pass a comparison function to the sorting algorithms, thereby making the algorithms polymorphic:

In [ ]:
man qsort

Function pointer declarations require careful use of parentheses.  The declaration
<code>
    double (*fun_pt)(double, double);
</code>
defines a pointer to a function with two `double` inputs and which returns a `double`.  On the other hand, the declaration
<code>
    double *fun_pt(double, double);
</code>
would be the function prototype for a function with two `double` inputs and which returns a pointer to a `double`.

As needed a function name will be implicitly converted to a function pointer.  Here we convert `pow()` from the C math library to a pointer:

In [ ]:
cat -n src/fun_pt.c

In [ ]:
gcc -Wall -pedantic src/fun_pt.c

In [ ]:
./a.out

# Dynamic memory allocation

In C you can dynamically allocate new objects in memory.  These are not to be confused with the **automatic variables**, variables that are local to functions that pass in and out of existence when functions are called.

In this discussion we will differentiate between two types of memory:
* [**stack memory**](https://en.wikipedia.org/wiki/Stack-based_memory_allocation), which is where things like automatic variables reside (along with a lot of other information needed by the program), and
* [**heap memory**](https://en.wikipedia.org/wiki/Memory_management#Dynamic_memory_allocation), which is where we can dynamically allocate memory whose contents persist.

Each computational process that is running on your computer has its own stack memory.  Because each process has a chunk of stack memory and there are a lot of processes, stack memory is limited in size.

Heap memory, on the other hand, is much more plentiful.

There is a family of functions in C you can use to dynamically allocate memory to create new arrays and other objects.  The most commonly encountered memory allocation function is called `malloc()`:

In [ ]:
cat src/malloc.txt

The function `malloc()` allocates a contiguous region of memory and returns a pointer to the beginning of the region.  **It does not initialize the allocated memory.**

The function `calloc()` does the same thing as `malloc()` but also clears the allocated region by initializing all the bytes to be zero.  Depending on the situation you might or might not want to incur this overhead (e.g., if you immediately write values to the allocated region there is no need to initialize it to zero).

<div class="danger"></div>
🐞 If you use <code>malloc()</code>, do not assume the allocated memory is full of anything except random bytes. 🐞

The return type of `malloc()` and `calloc()` is `void*`.  The `void*` type is C's way of indicating a generic memory address.  One typically converts (casts) the value returned by `malloc()` or `calloc()` to another type.  Here we use these functions to allocate arrays of length `n`:

In [ ]:
cat -n src/alloc.c

In [ ]:
clang -Wall -pedantic src/alloc.c

In [ ]:
./a.out

Notice the different calling sequences for `malloc()` and `calloc()`: `malloc()` takes as its input the total number of bytes that are to be allocated, while `calloc()` takes the number of objects (doubles, ints) and the size of the individual objects (in bytes).

If the memory allocation fails, say, because there is insufficient memory available, then these functions return the special value `nullptr`, [the null pointer](https://en.wikipedia.org/wiki/Null_pointer) (`NULL` prior to C23).  The null pointer is an invalid address (typically `0`) that no program should ever reference in the course of normal operation and attempting to access address `nullptr` will result in a runtime error:

In [ ]:
cat -n src/null.c

In [ ]:
clang -Wall -std=c23 src/null.c

In [ ]:
./a.out

Here is a common C idiom that attempts a memory allocation and checks for an error.  It uses an assignment operator:

In [ ]:
cat -n src/alloc1.c

In line 9 we first set `x = (double*) malloc(n * sizeof(double))`.  We then check whether `x == nullptr`.  If so, we print an error message.

The function `free()` takes a pointer as its input and frees the memory it points to.  It assumes that the pointer points to buffer (chunk of memory) allocated using one of C's memory allocation functions.  If not, you will encounter a runtime error:

In [ ]:
cat -n src/free.c

In [ ]:
clang -Wall src/free.c

In [ ]:
./a.out

C does not perform garbage collection &ndash; it is up to you to clean up and release memory when you are done with it.  If you do not, it is possible that you will run out of memory.  When we look at memory access errors we will look at a particular type of runtime error called a [memory leak](https://en.wikipedia.org/wiki/Memory_leak) which results from a failure to properly manage memory usage.

Typically all allocated memory is released when a program ends.  I say "typically" since I have worked with a stripped-down Unix operating system where this was not the case, and failure to explicitly call `free()` to release memory could cause the operating system to run out of memory and the system would crash.

# Stack overflow!

Variables that are local to functions are allocated on the stack.  However, the stack is limited in size (though C provides ways to resize it).

This means that if you have a local array that is too big to fit in the stack memory, then you'll crash.

In [ ]:
cat -n src/stack_overflow.c

In [ ]:
gcc src/stack_overflow.c

In [ ]:
./a.out

Variable length arrays also seem to have the same problem:

In [ ]:
cat -n src/vla.c

In [ ]:
gcc src/vla.c

In [ ]:
./a.out